In [78]:
from crypto_pandas import CCXTPandasExchange
import ccxt
import pandas as pd

pd.set_option('display.max_rows', 50)
pd.set_option('display.float_format', '{:.8f}'.format)
exchange = ccxt.upbit()
pandas_exchange = CCXTPandasExchange(exchange=exchange)
markets = pandas_exchange.load_markets()
markets = markets.query("quote == 'USDT'")

In [79]:
desired_depths = [1_000, 10_000, 100_000, 1_000_000, 5_000_000]

In [80]:
orderbook = pandas_exchange.fetchOrderBooks().merge(markets)
orderbook

,price,qty,symbol,timestamp,datetime,side,exchange,id,base,quote,...,option,index,active,contract,taker,maker,tierBased,percentage,precision_amount,precision_price
0,0.00134900,13491.52911849,ACS/USDT,2025-07-28 12:46:34.891000+00:00,2025-07-28 12:46:34.891000+00:00,asks,upbit,USDT-ACS,ACS,USDT,...,False,False,True,False,0.00250000,0.00250000,False,True,0.00000001,0.00000001
1,0.00135000,94728.47225592,ACS/USDT,2025-07-28 12:46:34.891000+00:00,2025-07-28 12:46:34.891000+00:00,asks,upbit,USDT-ACS,ACS,USDT,...,False,False,True,False,0.00250000,0.00250000,False,True,0.00000001,0.00000001
2,0.00139400,12661.84554103,ACS/USDT,2025-07-28 12:46:34.891000+00:00,2025-07-28 12:46:34.891000+00:00,asks,upbit,USDT-ACS,ACS,USDT,...,False,False,True,False,0.00250000,0.00250000,False,True,0.00000001,0.00000001
3,0.00139900,1406.87172678,ACS/USDT,2025-07-28 12:46:34.891000+00:00,2025-07-28 12:46:34.891000+00:00,asks,upbit,USDT-ACS,ACS,USDT,...,False,False,True,False,0.00250000,0.00250000,False,True,0.00000001,0.00000001
4,0.00140000,714.28571428,ACS/USDT,2025-07-28 12:46:34.891000+00:00,2025-07-28 12:46:34.891000+00:00,asks,upbit,USDT-ACS,ACS,USDT,...,False,False,True,False,0.00250000,0.00250000,False,True,0.00000001,0.00000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6895,0.10070000,134.75062900,ZRX/USDT,2025-07-28 13:27:04.251000+00:00,2025-07-28 13:27:04.251000+00:00,bids,upbit,USDT-ZRX,ZRX,USDT,...,False,False,True,False,0.00250000,0.00250000,False,True,0.00000001,0.00000001
6896,0.10060000,500.00000000,ZRX/USDT,2025-07-28 13:27:04.251000+00:00,2025-07-28 13:27:04.251000+00:00,bids,upbit,USDT-ZRX,ZRX,USDT,...,False,False,True,False,0.00250000,0.00250000,False,True,0.00000001,0.00000001
6897,0.10020000,5.46307900,ZRX/USDT,2025-07-28 13:27:04.251000+00:00,2025-07-28 13:27:04.251000+00:00,bids,upbit,USDT-ZRX,ZRX,USDT,...,False,False,True,False,0.00250000,0.00250000,False,True,0.00000001,0.00000001
6898,0.06002000,9.41623705,ZRX/USDT,2025-07-28 13:27:04.251000+00:00,2025-07-28 13:27:04.251000+00:00,bids,upbit,USDT-ZRX,ZRX,USDT,...,False,False,True,False,0.00250000,0.00250000,False,True,0.00000001,0.00000001


In [81]:
# Calculate notional, cum depth, and max depth

orderbook["notional"] = orderbook["price"]*orderbook["qty"]
orderbook["cumNotional"] = orderbook.groupby(["symbol", "exchange", "side"])["notional"].cumsum()
orderbook["totalNotional"] = orderbook.groupby(["symbol", "exchange", "side"])["notional"].transform("sum")
orderbook["shiftCumNotional"] = orderbook["cumNotional"] - orderbook["notional"]
orderbook

,price,qty,symbol,timestamp,datetime,side,exchange,id,base,quote,...,taker,maker,tierBased,percentage,precision_amount,precision_price,notional,cumNotional,totalNotional,shiftCumNotional
0,0.00134900,13491.52911849,ACS/USDT,2025-07-28 12:46:34.891000+00:00,2025-07-28 12:46:34.891000+00:00,asks,upbit,USDT-ACS,ACS,USDT,...,0.00250000,0.00250000,False,True,0.00000001,0.00000001,18.20007278,18.20007278,4265.51432755,0.00000000
1,0.00135000,94728.47225592,ACS/USDT,2025-07-28 12:46:34.891000+00:00,2025-07-28 12:46:34.891000+00:00,asks,upbit,USDT-ACS,ACS,USDT,...,0.00250000,0.00250000,False,True,0.00000001,0.00000001,127.88343755,146.08351033,4265.51432755,18.20007278
2,0.00139400,12661.84554103,ACS/USDT,2025-07-28 12:46:34.891000+00:00,2025-07-28 12:46:34.891000+00:00,asks,upbit,USDT-ACS,ACS,USDT,...,0.00250000,0.00250000,False,True,0.00000001,0.00000001,17.65061268,163.73412301,4265.51432755,146.08351033
3,0.00139900,1406.87172678,ACS/USDT,2025-07-28 12:46:34.891000+00:00,2025-07-28 12:46:34.891000+00:00,asks,upbit,USDT-ACS,ACS,USDT,...,0.00250000,0.00250000,False,True,0.00000001,0.00000001,1.96821355,165.70233656,4265.51432755,163.73412301
4,0.00140000,714.28571428,ACS/USDT,2025-07-28 12:46:34.891000+00:00,2025-07-28 12:46:34.891000+00:00,asks,upbit,USDT-ACS,ACS,USDT,...,0.00250000,0.00250000,False,True,0.00000001,0.00000001,1.00000000,166.70233656,4265.51432755,165.70233656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6895,0.10070000,134.75062900,ZRX/USDT,2025-07-28 13:27:04.251000+00:00,2025-07-28 13:27:04.251000+00:00,bids,upbit,USDT-ZRX,ZRX,USDT,...,0.00250000,0.00250000,False,True,0.00000001,0.00000001,13.56938834,15992.50534011,16044.58632345,15978.93595177
6896,0.10060000,500.00000000,ZRX/USDT,2025-07-28 13:27:04.251000+00:00,2025-07-28 13:27:04.251000+00:00,bids,upbit,USDT-ZRX,ZRX,USDT,...,0.00250000,0.00250000,False,True,0.00000001,0.00000001,50.30000000,16042.80534011,16044.58632345,15992.50534011
6897,0.10020000,5.46307900,ZRX/USDT,2025-07-28 13:27:04.251000+00:00,2025-07-28 13:27:04.251000+00:00,bids,upbit,USDT-ZRX,ZRX,USDT,...,0.00250000,0.00250000,False,True,0.00000001,0.00000001,0.54740052,16043.35274063,16044.58632345,16042.80534011
6898,0.06002000,9.41623705,ZRX/USDT,2025-07-28 13:27:04.251000+00:00,2025-07-28 13:27:04.251000+00:00,bids,upbit,USDT-ZRX,ZRX,USDT,...,0.00250000,0.00250000,False,True,0.00000001,0.00000001,0.56516255,16043.91790317,16044.58632345,16043.35274063


In [82]:
# Calculate orderbook VWAPS

vwaps = []
for depth in desired_depths:
    vwap = orderbook.query(f"shiftCumNotional <= {depth}").copy()
    vwap["notional"] = vwap["notional"].clip(upper=vwap["cumNotional"] - depth)
    vwap["depth"] = depth
    vwaps.append(vwap.copy())
vwaps = pd.concat(vwaps)
vwaps["qty"] = vwaps["notional"]/vwaps["price"]
vwaps = vwaps.groupby(["symbol", "exchange", "side", "depth"], as_index=False).agg(
    totalNotional=("totalNotional", "sum"),
    notional=("notional", "sum"),
    qty=("qty", "sum"),
)
vwaps["vwap"] = vwaps["notional"]/vwaps["qty"]
vwaps = vwaps.query("totalNotional >= depth")

In [86]:
# Calculate orderbook depth spreads
vwaps_pivot = vwaps.pivot(index=["symbol", "exchange", "depth"], columns="side", values="vwap").reset_index()
vwaps_pivot["spread"] = vwaps_pivot["asks"] - vwaps_pivot["bids"]
vwaps_pivot["spreadRelative"] = vwaps_pivot["spread"]/vwaps_pivot[["asks", "bids"]].mean(axis=1)
vwaps_pivot

side,symbol,exchange,depth,asks,bids,spread,spreadRelative
0,ACS/USDT,upbit,1000,0.00145425,0.00125694,0.00019730,0.14554753
1,ACS/USDT,upbit,10000,0.00170150,0.00100893,0.00069257,0.51104122
2,ACS/USDT,upbit,100000,0.00174319,0.00097367,0.00076952,0.56648008
3,ADA/USDT,upbit,1000,0.97221010,0.85593654,0.11627355,0.12720375
4,ADA/USDT,upbit,10000,1.03363268,0.79568424,0.23794844,0.26015004
...,...,...,...,...,...,...,...
391,ZRO/USDT,upbit,10000,2.55786230,1.89792068,0.65994162,0.29621803
392,ZRO/USDT,upbit,100000,NaN,1.07400710,NaN,NaN
393,ZRX/USDT,upbit,1000,0.31800000,0.26872861,0.04927139,0.16795290
394,ZRX/USDT,upbit,10000,0.37721271,0.26122065,0.11599206,0.36336465


In [84]:
vwaps_pivot.query("symbol == 'BTC/USDT'")

side,symbol,exchange,depth,asks,bids,spread,spreadRelative
64,BTC/USDT,upbit,1000,119041.42741566,117999.67535112,1041.75206455,0.00878963
65,BTC/USDT,upbit,10000,119040.00754159,118058.27167423,981.73586735,0.00828126
66,BTC/USDT,upbit,100000,119440.01285276,117601.14521890,1838.86763385,0.01551518
67,BTC/USDT,upbit,1000000,119526.16259103,117566.49854816,1959.66404287,0.01653079
